In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important;} *{font-family:'Consolas'; font-size:14;} </style>"))

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

# 머신러닝 라이브러리를 이용한 분류

In [44]:
import numpy as np
import pandas as pd
from sklearn import datasets

In [8]:
iris = datasets.load_iris()
iris_X = iris.data
iris_y = pd.get_dummies(iris.target).to_numpy()

In [12]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(iris_X, iris_y, test_size=0.3,
                                                    random_state=1)

In [14]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes = (50,50,30))

In [15]:
mlp.fit(train_X, train_y)

C:\Users\COM\AppData\Roaming\Python\Python37\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(50, 50, 30), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [16]:
mlp.score(test_X, test_y)

0.9777777777777777

# 텐서플로우를 이용한 분류

In [26]:
import tensorflow as tf

In [27]:
# 텐서플로우 2.x 이상일 경우
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [28]:
# 입력한 값을 저장할 Tensor
# [행의 수, 열의 수] : None인 경우 아직 알 수 없음(지정하지 않음)
x = tf.placeholder(tf.float32, [None, 4])
y = tf.placeholder(tf.float32, [None, 3])

In [29]:
# define Weight & Bias
W = tf.Variable(tf.zeros([4, 3]))
b = tf.Variable(tf.zeros([3]))

In [30]:
h = tf.nn.softmax(tf.matmul(x, W)+b)

In [31]:
# cross_entropy도 Tensor
cross_entropy = -tf.reduce_sum(y * tf.log(h),
                               reduction_indices=[1])

In [32]:
loss = tf.reduce_mean(cross_entropy)

In [33]:
train = tf.train.GradientDescentOptimizer(0.001).minimize(loss)

In [34]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [35]:
for i in range(100001):
    _, loss_value = sess.run([train, loss], feed_dict={x: train_X, y: train_y})
    if i % 10000 == 0:
        print("Step %d, %.5f" % (i, loss_value))

Step 0, 1.09861
Step 10000, 0.34161
Step 20000, 0.25734
Step 30000, 0.21406
Step 40000, 0.18753
Step 50000, 0.16957
Step 60000, 0.15658
Step 70000, 0.14673
Step 80000, 0.13899
Step 90000, 0.13274
Step 100000, 0.12758


In [38]:
correct_prediction = tf.equal(tf.argmax(h, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x:test_X, y:test_y}))

0.9777778


In [39]:
print(sess.run(h, feed_dict={x:test_X, y:test_y})[0:10])

[[9.98785913e-01 1.21412973e-03 1.14380692e-10]
 [1.21662505e-01 8.65827620e-01 1.25099365e-02]
 [1.11910831e-02 9.52915967e-01 3.58929113e-02]
 [9.97843385e-01 2.15664413e-03 1.68622261e-09]
 [1.24174898e-04 3.19967151e-01 6.79908633e-01]
 [7.99214933e-03 7.64345944e-01 2.27661923e-01]
 [4.96162567e-04 2.62597650e-01 7.36906230e-01]
 [9.86688554e-01 1.33113479e-02 1.60573222e-07]
 [9.74976122e-01 2.50234734e-02 3.16451974e-07]
 [2.89404798e-05 5.79523407e-02 9.42018747e-01]]


In [40]:
predict_y = sess.run(tf.argmax(h, 1), feed_dict={x:test_X})

In [41]:
print(predict_y[0:20])

[0 1 1 0 2 1 2 0 0 2 1 0 2 1 1 0 1 1 0 0]


In [42]:
print(test_y.argmax(axis=1)[0:20])

[0 1 1 0 2 1 2 0 0 2 1 0 2 1 1 0 1 1 0 0]


In [44]:
cross_tab = pd.crosstab(test_y.argmax(axis=1), predict_y, margins=True)
cross_tab

col_0,0,1,2,All
row_0,,,,
0,14,0,0,14
1,0,17,1,18
2,0,0,13,13
All,14,17,14,45


# 케라스를 이용한 분류

In [46]:
iris = datasets.load_iris()
iris_X = iris.data
iris_y = pd.get_dummies(iris.target).to_numpy()

In [47]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(iris_X, iris_y, test_size=0.3,
                                                    random_state=1)

In [48]:
train_X.shape, train_y.shape

((105, 4), (105, 3))

In [50]:
from tensorflow.keras.models import Sequential
model = Sequential()

In [51]:
from tensorflow.keras.layers import Input, Dense
model.add(Input(4))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(3, activation='softmax'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [53]:
model.compile(optimizer="adam", loss="categorical_crossentropy", 
              metrics=["accuracy"])

In [54]:
model.fit(train_X, train_y, batch_size=50, epochs=200, verbose=1)

Train on 105 samples
Epoch 1/200
105/105 [==============================] - 0s 495us/sample - loss: 1.1168 - acc: 0.3524
Epoch 2/200
105/105 [==============================] - 0s 38us/sample - loss: 1.0443 - acc: 0.3524
Epoch 3/200
105/105 [==============================] - 0s 38us/sample - loss: 0.9955 - acc: 0.3810
Epoch 4/200
105/105 [==============================] - 0s 38us/sample - loss: 0.9609 - acc: 0.6857
Epoch 5/200
105/105 [==============================] - 0s 29us/sample - loss: 0.9339 - acc: 0.6952
Epoch 6/200
105/105 [==============================] - 0s 48us/sample - loss: 0.9038 - acc: 0.6952
Epoch 7/200
105/105 [==============================] - 0s 67us/sample - loss: 0.8772 - acc: 0.6952
Epoch 8/200
105/105 [==============================] - 0s 29us/sample - loss: 0.8548 - acc: 0.6952
Epoch 9/200
105/105 [==============================] - 0s 48us/sample - loss: 0.8296 - acc: 0.6952
Epoch 10/200
105/105 [==============================] - 0s 48us/sample - loss: 0.8033 -

105/105 [==============================] - 0s 29us/sample - loss: 0.0795 - acc: 0.9714
Epoch 164/200
105/105 [==============================] - 0s 38us/sample - loss: 0.0863 - acc: 0.9714
Epoch 165/200
105/105 [==============================] - 0s 29us/sample - loss: 0.0811 - acc: 0.9714
Epoch 166/200
105/105 [==============================] - 0s 38us/sample - loss: 0.0844 - acc: 0.9714
Epoch 167/200
105/105 [==============================] - 0s 38us/sample - loss: 0.0912 - acc: 0.9714
Epoch 168/200
105/105 [==============================] - 0s 29us/sample - loss: 0.0834 - acc: 0.9619
Epoch 169/200
105/105 [==============================] - 0s 38us/sample - loss: 0.0808 - acc: 0.9619
Epoch 170/200
105/105 [==============================] - 0s 57us/sample - loss: 0.1024 - acc: 0.9619
Epoch 171/200
105/105 [==============================] - 0s 38us/sample - loss: 0.0909 - acc: 0.9714
Epoch 172/200
105/105 [==============================] - 0s 29us/sample - loss: 0.0838 - acc: 0.9714
Epoc

In [56]:
pred = np.argmax(model.predict(test_X), axis=1)
pred

array([0, 1, 1, 0, 2, 1, 2, 0, 0, 2, 1, 0, 2, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 0, 2, 1, 0, 0, 1, 2, 1, 2, 1, 2, 2, 0, 1, 0, 1, 2, 2, 0, 2, 2,
       1], dtype=int64)

In [59]:
np.argmax(test_y, axis=1) # 테스트 데이터의 정답

array([0, 1, 1, 0, 2, 1, 2, 0, 0, 2, 1, 0, 2, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 0, 2, 1, 0, 0, 1, 2, 1, 2, 1, 2, 2, 0, 1, 0, 1, 2, 2, 0, 2, 2,
       1], dtype=int64)

In [60]:
pd.crosstab(np.argmax(test_y, axis=1), pred)

col_0,0,1,2
row_0,,,
0,14,0,0
1,0,18,0
2,0,0,13


In [61]:
model.evaluate(test_X, test_y)

45/45 [==============================] - 0s 200us/sample - loss: 0.0462 - acc: 1.0000


[0.04622440002858639, 1.0]

# Redwine 데이터 분류하기

## Tensorflow

In [63]:
import numpy as np
import pandas as pd

In [81]:
redwine = pd.read_csv("winequality-red.csv", sep=';')
redwine_X = redwine.iloc[:, :-1].to_numpy()
redwine_y = pd.get_dummies(redwine.iloc[:, -1]).to_numpy()

In [82]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(redwine_X, redwine_y, test_size=0.3, random_state=1)

In [83]:
# 텐서플로우 2.x 이상일 경우
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [84]:
x = tf.placeholder(tf.float32, [None, 11])
y = tf.placeholder(tf.float32, [None, 6])

# placeholder: Tensor를 할당  
# tf.placeholder(dtype, shape=None, name=None)

In [85]:
# define Weight & Bias
W = tf.Variable(tf.zeros([11, 6]))
b = tf.Variable(tf.zeros([6]))

# Variable: 모델을 훈련할 때, 모델 파라미터들을 저장할 변수들
# 텐서를 메모리에 저장

In [86]:
h = tf.nn.softmax(tf.matmul(x, W)+b)

In [87]:
# cross_entropy도 Tensor
cross_entropy = -tf.reduce_sum(y * tf.log(h),
                               reduction_indices=[1])

In [88]:
loss = tf.reduce_mean(cross_entropy)

In [89]:
train = tf.train.GradientDescentOptimizer(0.001).minimize(loss)

In [90]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [98]:
for i in range(1000001):
    _, loss_value = sess.run([train, loss], feed_dict={x: train_X, y: train_y})
    if i % 10000 == 0:
        print("Step %d, %.5f" % (i, loss_value))

Step 0, 0.98717
Step 10000, 0.98460
Step 20000, 0.98234
Step 30000, 0.98032
Step 40000, 0.97848
Step 50000, 0.97679
Step 60000, 0.97523
Step 70000, 0.97378
Step 80000, 0.97241
Step 90000, 0.97113
Step 100000, 0.96991
Step 110000, 0.96876
Step 120000, 0.96765
Step 130000, 0.96660
Step 140000, 0.96560
Step 150000, 0.96463
Step 160000, 0.96371
Step 170000, 0.96281
Step 180000, 0.96196
Step 190000, 0.96113
Step 200000, 0.96033
Step 210000, 0.95956
Step 220000, 0.95882
Step 230000, 0.95809
Step 240000, 0.95739
Step 250000, 0.95672
Step 260000, 0.95607
Step 270000, 0.95543
Step 280000, 0.95481
Step 290000, 0.95421
Step 300000, 0.95363
Step 310000, 0.95306
Step 320000, 0.95251
Step 330000, 0.95197
Step 340000, 0.95144
Step 350000, 0.95093
Step 360000, 0.95043
Step 370000, 0.94995
Step 380000, 0.94948
Step 390000, 0.94902
Step 400000, 0.94857
Step 410000, 0.94813
Step 420000, 0.94769
Step 430000, 0.94728
Step 440000, 0.94688
Step 450000, 0.94648
Step 460000, 0.94608
Step 470000, 0.94570
Step 4

In [99]:
correct_prediction = tf.equal(tf.argmax(h, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x:test_X, y:test_y}))

0.59791666


In [100]:
predict_y = sess.run(tf.argmax(h, 1), feed_dict={x:test_X})

In [101]:
cross_tab = pd.crosstab(test_y.argmax(axis=1), predict_y, margins=True)
cross_tab

col_0,2,3,4,All
row_0,,,,
0,2,0,0,2
1,12,8,1,21
2,156,49,2,207
3,63,117,15,195
4,4,34,14,52
5,0,1,2,3
All,237,209,34,480


## Keras

In [102]:
redwine = pd.read_csv("winequality-red.csv", sep=';')
redwine_X = redwine.iloc[:, :-1].to_numpy()
redwine_y = pd.get_dummies(redwine.iloc[:, -1]).to_numpy()

In [103]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(redwine_X, redwine_y, test_size=0.3, random_state=1)

In [104]:
train_X.shape, train_y.shape

((1119, 11), (1119, 6))

In [109]:
from tensorflow.keras.models import Sequential
model = Sequential()

In [110]:
from tensorflow.keras.layers import Input, Dense
model.add(Input(11))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(6, activation='softmax'))

In [111]:
model.compile(optimizer="adam", loss="categorical_crossentropy", 
              metrics=["accuracy"])

In [112]:
model.fit(train_X, train_y, batch_size=50, epochs=200, verbose=1)

Train on 1119 samples
Epoch 1/200
1119/1119 [==============================] - 0s 57us/sample - loss: 1.5122 - acc: 0.4173
Epoch 2/200
1119/1119 [==============================] - 0s 17us/sample - loss: 1.1933 - acc: 0.4906
Epoch 3/200
1119/1119 [==============================] - 0s 19us/sample - loss: 1.1638 - acc: 0.4861
Epoch 4/200
1119/1119 [==============================] - 0s 20us/sample - loss: 1.1474 - acc: 0.5049
Epoch 5/200
1119/1119 [==============================] - 0s 18us/sample - loss: 1.1385 - acc: 0.4951
Epoch 6/200
1119/1119 [==============================] - 0s 21us/sample - loss: 1.1267 - acc: 0.5183
Epoch 7/200
1119/1119 [==============================] - 0s 21us/sample - loss: 1.1191 - acc: 0.5237
Epoch 8/200
1119/1119 [==============================] - 0s 19us/sample - loss: 1.1198 - acc: 0.5174
Epoch 9/200
1119/1119 [==============================] - 0s 20us/sample - loss: 1.1177 - acc: 0.5273
Epoch 10/200
1119/1119 [==============================] - 0s 19us/sam

1119/1119 [==============================] - 0s 18us/sample - loss: 0.7965 - acc: 0.6765
Epoch 161/200
1119/1119 [==============================] - 0s 17us/sample - loss: 0.8189 - acc: 0.6568
Epoch 162/200
1119/1119 [==============================] - 0s 17us/sample - loss: 0.7938 - acc: 0.6693
Epoch 163/200
1119/1119 [==============================] - 0s 17us/sample - loss: 0.8049 - acc: 0.6524
Epoch 164/200
1119/1119 [==============================] - 0s 17us/sample - loss: 0.7900 - acc: 0.6801
Epoch 165/200
1119/1119 [==============================] - 0s 17us/sample - loss: 0.8054 - acc: 0.6702
Epoch 166/200
1119/1119 [==============================] - 0s 17us/sample - loss: 0.8024 - acc: 0.6693
Epoch 167/200
1119/1119 [==============================] - 0s 16us/sample - loss: 0.7953 - acc: 0.6765
Epoch 168/200
1119/1119 [==============================] - 0s 16us/sample - loss: 0.8112 - acc: 0.6693
Epoch 169/200
1119/1119 [==============================] - 0s 16us/sample - loss: 0.786

In [113]:
pred = np.argmax(model.predict(test_X), axis=1)

In [114]:
pd.crosstab(np.argmax(test_y, axis=1), pred)

col_0,1,2,3,4
row_0,,,,
0,1,1,0,0
1,0,17,3,1
2,0,162,41,4
3,0,66,118,11
4,0,6,29,17
5,0,1,0,2


In [115]:
model.evaluate(test_X, test_y) # loss, accuracy

480/480 [==============================] - 0s 75us/sample - loss: 1.1147 - acc: 0.6187


[1.1147265553474426, 0.61875]

# 숫자 이미지 분류하기

## 머신러닝

In [3]:
import cv2
import numpy as np

img = cv2.imread("images/digits.png", cv2.IMREAD_GRAYSCALE)
print(img.shape)

(1000, 2000)


In [118]:
cv2.imshow("number", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [119]:
cells = [np.hsplit(row, 100) for row in np.vsplit(img, 50)]
x = np.array(cells)

In [120]:
x.shape

(50, 100, 20, 20)

In [121]:
X = x[:,:].reshape(-1, 400).astype(np.float32)
y = np.repeat(np.arange(10), 500)

In [122]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=1)

In [124]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(512, 256, 128), verbose=True)

In [125]:
mlp.fit(train_X, train_y)

Iteration 1, loss = 17.11732132
Iteration 2, loss = 1.49612161
Iteration 3, loss = 0.64257926
Iteration 4, loss = 0.30388067
Iteration 5, loss = 0.14290359
Iteration 6, loss = 0.05101895
Iteration 7, loss = 0.02703091
Iteration 8, loss = 0.01008356
Iteration 9, loss = 0.00931235
Iteration 10, loss = 0.00879675
Iteration 11, loss = 0.00529933
Iteration 12, loss = 0.00350962
Iteration 13, loss = 0.00184601
Iteration 14, loss = 0.00067759
Iteration 15, loss = 0.00052721
Iteration 16, loss = 0.00047438
Iteration 17, loss = 0.00044835
Iteration 18, loss = 0.00043110
Iteration 19, loss = 0.00041599
Iteration 20, loss = 0.00040456
Iteration 21, loss = 0.00039483
Iteration 22, loss = 0.00038614
Iteration 23, loss = 0.00037811
Iteration 24, loss = 0.00037162
Iteration 25, loss = 0.00036526
Iteration 26, loss = 0.00035952
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(512, 256, 128), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=True,
              warm_start=False)

In [126]:
mlp.score(test_X, test_y)

0.9026666666666666

In [ ]:
mlp.predict(proba=)

## Tensorflow

In [9]:
cells = [np.hsplit(row,100) for row in np.vsplit(img, 50)]
x = np.array(cells)
X = x[:,:].reshape(-1,400).astype(np.float32)
y = pd.get_dummies(np.repeat(np.arange(10), 500)).to_numpy()

In [10]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=1)

In [11]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [12]:
x = tf.placeholder(tf.float32, [None, 400])
y = tf.placeholder(tf.float32, [None, 10])

In [13]:
W1 = tf.Variable(tf.zeros([400, 512]))
b1 = tf.Variable(tf.zeros([512]))
h1 = tf.nn.relu(tf.matmul(x, W1)+b1)

In [14]:
W2 = tf.Variable(tf.zeros([512, 256]))
b2 = tf.Variable(tf.zeros([256]))
h2 = tf.nn.relu(tf.matmul(h1, W2)+b2)

In [15]:
W3 = tf.Variable(tf.zeros([256, 128]))
b3 = tf.Variable(tf.zeros([128]))
h3 = tf.nn.relu(tf.matmul(h2, W3)+b3)

In [16]:
W = tf.Variable(tf.zeros([128, 10]))
b = tf.Variable(tf.zeros([10]))

In [17]:
h = tf.nn.softmax(tf.matmul(h3, W)+b)

In [18]:
cross_entropy = -tf.reduce_sum(y * tf.log(h), reduction_indices=[1])

In [19]:
loss = tf.reduce_mean(cross_entropy)

In [20]:
train = tf.train.GradientDescentOptimizer(0.001).minimize(loss)

In [21]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [146]:
for i in range(100001):
    _, loss_value = sess.run([train, loss], feed_dict={x:train_X, y:train_y})
    if i % 10000 == 0:
        print("Step %d, %.5f" % (i, loss_value))

Step 0, 2.30260
Step 10000, 2.30214
Step 20000, 2.30208
Step 30000, 2.30205
Step 40000, 2.30206
Step 50000, 2.30207
Step 60000, 2.30207
Step 70000, 2.30206
Step 80000, 2.30206
Step 90000, 2.30207
Step 100000, 2.30207


In [147]:
correct_prediction = tf.equal(tf.argmax(h, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x:test_X, y:test_y}))

0.086666666


In [148]:
predict_y = sess.run(tf.argmax(h, 1), feed_dict={x:test_X})

In [151]:
predict_y[0:20]

array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
      dtype=int64)

In [149]:
cross_tab = pd.crosstab(test_y.argmax(axis=1), predict_y, margins=True)
cross_tab

col_0,4,All
row_0,,
0,155,155
1,149,149
2,152,152
3,171,171
4,130,130
5,148,148
6,135,135
7,160,160
8,156,156


## Tensorflow + 

In [ ]:
cells = [np.hsplit(row,100) for row in np.vsplit(img, 50)]
x = np.array(cells)
X = x[:,:].reshape(-1,400).astype(np.float32)
y = pd.get_dummies(np.repeat(np.arange(10), 500)).to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [ ]:
x = tf.placeholder(tf.float32, [None, 400])
y = tf.placeholder(tf.float32, [None, 10])

In [ ]:
W1 = tf.Variable(tf.zeros([400, 512]))
b1 = tf.Variable(tf.zeros([512]))
h1 = tf.nn.relu(tf.matmul(x, W1)+b1)

In [ ]:
W2 = tf.Variable(tf.zeros([512, 256]))
b2 = tf.Variable(tf.zeros([256]))
h2 = tf.nn.relu(tf.matmul(h1, W2)+b2)

In [ ]:
W3 = tf.Variable(tf.zeros([256, 128]))
b3 = tf.Variable(tf.zeros([128]))
h3 = tf.nn.relu(tf.matmul(h2, W3)+b3)

In [ ]:
W = tf.Variable(tf.zeros([128, 10]))
b = tf.Variable(tf.zeros([10]))

In [ ]:
h = tf.nn.softmax(tf.matmul(h3, W)+b)

In [ ]:
cross_entropy = -tf.reduce_sum(y * tf.log(h), reduction_indices=[1])

In [ ]:
loss = tf.reduce_mean(cross_entropy)

In [ ]:
train = tf.train.GradientDescentOptimizer(0.001).minimize(loss)

In [ ]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [22]:
tot_cnt = len(train_X)

for epoch in range(1000):
    avg_loss = 0
    batch_size = 128
    tot_iter = int(tot_cnt / batch_size)
    for step in range(tot_iter):
        batch_X = train_X[int(step*batch_size):int((step+1)*batch_size)]
        batch_y = train_y[int(step*batch_size):int((step+1)*batch_size)]
        _, loss_ = sess.run([train, loss], feed_dict={x:batch_X, y:batch_y})
        avg_loss += (loss_ / tot_iter)
        print("{:03d} - {:f}".format(step, loss_), end="\r")
    if tot_cnt//tot_iter != 0:
        batch_X = train_X[int(tot_iter*batch_size):]
        batch_y = train_y[int(tot_iter*batch_size):]
        _, loss_ = sess.run([train, loss], feed_dict={x:batch_X, y:batch_y})
        avg_loss += (loss_ / tot_iter)
        print("{:03d} - {:f}".format(step, loss_), end="\r")
    if epoch%100 == 0:
        print("step {}, train loss {}".format(epoch, avg_loss))

step 0, train loss 2.387868996019716
step 100, train loss 2.3876843099240905
step 200, train loss 2.3875784697356046
step 300, train loss 2.387517522882532
step 400, train loss 2.3874825371636286
step 500, train loss 2.387462271584405
step 600, train loss 2.3874505979043468
step 700, train loss 2.3874437455777775
step 800, train loss 2.3874398778986046
step 900, train loss 2.387437582015991


In [23]:
correct_prediction = tf.equal(tf.argmax(h, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x:test_X, y:test_y}))

0.086666666


In [24]:
sess.close()

## Gradient Vanishing 문제 해결을 위해

In [ ]:
import numpy as np
import pandas as pd

In [25]:
cells = [np.hsplit(row,100) for row in np.vsplit(img, 50)]
x = np.array(cells) / 255
X = x[:,:].reshape(-1,400).astype(np.float32)
y = pd.get_dummies(np.repeat(np.arange(10), 500)).to_numpy()

In [26]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=1)

In [27]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [28]:
x = tf.placeholder(tf.float32, [None, 400])
y = tf.placeholder(tf.float32, [None, 10])

In [29]:
W1 = tf.Variable(tf.truncated_normal([400, 512], stddev=0.1))
b1 = tf.Variable(tf.constant(0.1, shape=[512]))
h1 = tf.nn.relu(tf.matmul(x, W1)+b1)

In [30]:
W2 = tf.Variable(tf.truncated_normal([512, 256], stddev=0.1))
b2 = tf.Variable(tf.constant(0.1, shape=[256]))
h2 = tf.nn.relu(tf.matmul(h1, W2)+b2)

In [31]:
# 너무 느려서 layer 수 줄였음 

In [32]:
W = tf.Variable(tf.zeros([256, 10]))
b = tf.Variable(tf.zeros([10]))

In [34]:
h = tf.nn.softmax(tf.matmul(h2, W)+b)

In [35]:
cross_entropy = -tf.reduce_sum(y * tf.log(h), reduction_indices=[1])

In [36]:
loss = tf.reduce_mean(cross_entropy)

In [37]:
train = tf.train.GradientDescentOptimizer(0.5).minimize(loss) 
# 학습률 변경: 0.001 > 0.5
# 크게 해도 상관은 없음

In [38]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [39]:
tot_cnt = len(train_X)

for epoch in range(1000):
    avg_loss = 0
    batch_size = 128
    tot_iter = int(tot_cnt / batch_size)
    for step in range(tot_iter):
        batch_X = train_X[int(step*batch_size):int((step+1)*batch_size)]
        batch_y = train_y[int(step*batch_size):int((step+1)*batch_size)]
        _, loss_ = sess.run([train, loss], feed_dict={x:batch_X, y:batch_y})
        avg_loss += (loss_ / tot_iter)
        print("{:03d} - {:f}".format(step, loss_), end="\r")
    if tot_cnt//tot_iter != 0:
        batch_X = train_X[int(tot_iter*batch_size):]
        batch_y = train_y[int(tot_iter*batch_size):]
        _, loss_ = sess.run([train, loss], feed_dict={x:batch_X, y:batch_y})
        avg_loss += (loss_ / tot_iter)
        print("{:03d} - {:f}".format(step, loss_), end="\r")
    if epoch%100 == 0:
        print("step {}, train loss {}".format(epoch, avg_loss))

step 0, train loss 1.4903600789882523
step 100, train loss 0.0005546786369113424
step 200, train loss 0.00023321167369700827
step 300, train loss 0.0001425827809995368
step 400, train loss 0.00010100715897149508
step 500, train loss 7.747906245101519e-05
step 600, train loss 6.246288042651335e-05
step 700, train loss 5.209508856717515e-05
step 800, train loss 4.454168412040518e-05
step 900, train loss 3.881289005760293e-05


In [40]:
correct_prediction = tf.equal(tf.argmax(h, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x:test_X, y:test_y}))

0.948


In [41]:
sess.close() # 메모리를 잡아먹기 때문에 session은 종료할 것

# CNN을 이용한 MNIST 데이터 모델링의 코드와 실행

In [6]:
import numpy as np
import pandas as pd
from sklearn import datasets

## 입력 데이터

In [7]:
import cv2
img = cv2.imread("images/digits.png", cv2.IMREAD_GRAYSCALE)

In [8]:
cells = [np.hsplit(row,100) for row in np.vsplit(img, 50)]
x = np.array(cells) / 255
X = x[:,:].reshape(-1,400).astype(np.float32)
y = pd.get_dummies(np.repeat(np.arange(10), 500)).to_numpy()

In [9]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=1)

## 레이어 정의

In [11]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [12]:
x = tf.placeholder(tf.float32, [None, 400])
y = tf.placeholder(tf.float32, [None, 10])
x_image = tf.reshape(x, [-1, 20, 20, 1]) # 행의 수: -1(남은 수를 할당), images:20*20 , channel수:1

In [13]:
# convolution layer의 weight = filter
# > filter의 shape대로 weight를 만들어야 함
# DNN의 Weight는 스칼라값 

# 3*3, channel이 1개인 32개의 뉴런을 만듦
W_conv1 = tf.Variable(tf.truncated_normal([3,3,1,32], stddev=0.1)) # Weight
# b_conv1 = tf.constant(0.1, shape=[32]) # bias. 넣어도 되고 안넣어도 됨
# bias의 shape는 뉴런의 수와 같아야 함(32) = 필터의 수

# Weight(W_conv1)와 입력값(x_image) 사이에 합성곱이 이루어져야 함
# > tensorflow.nn.conv2d 함수가 이를 수행
h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1, strides=[1,1,1,1], padding='SAME'))# + b_conv1)
# stride: 감소시킨 화소에 따라 stride 값이 달라짐
# 화소를 1/2로 줄임: stride=2
# 화소를 1/3로 줄임: stride=3
# stride=[안씀, x방향, y방향, 안씀] (0,3 index는 확장성을 위해 사용하는 것. 우선은 쓰지 않음)
h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
# ksize를 2씩 줄였으니 stride도 2로
h_pool1_drop = tf.nn.dropout(h_pool1, rate=0.1)
# 뉴런이 너무 많아서 학습이 너무 잘돼서..덜 하라고 dropout(Overfitting 방지)
# :10%의 확률로 Weight가 조정이 되지 않음
# 최대 40%까지 dropout. 물론 절대적인 수치는 없음

In [14]:
W_conv2 = tf.Variable(tf.truncated_normal([3,3,32,64], stddev=0.1))
# 이전 레이어의 뉴런이 32개였음: 3*3 image가 32개가 들어옴
# 현재 레이어의 뉴런의 수는 64
# 32, 64 등의 뉴런의 수는 연구자가 임의로 지정해야 하는 수. 당연히 뉴런이 많으면 실행 속도가 느리므로, 적당한 수치를 찾아내야 함
# b_conv2 = tf.constant(0.1, shape=[64])

h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1_drop, W_conv2, strides=[1,1,1,1], padding='SAME'))# + b_conv2)
h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
# 2*2로 pooling, 화소가 모자라면 한개 채우기

h_pool2_drop = tf.nn.dropout(h_pool2, rate=0.25)

In [15]:
W_fc1 = tf.Variable(tf.truncated_normal([5*5*64, 1024], stddev=0.1)) # 5*5는 이미지 따라 바꿔야 함
# convolution에 따라 나온 이미지 크기가 5*5. 그게 64개
# 은닉층 뉴런의 수가 1024개
# b_fc1 = tf.constant(0.1, shape=[1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 5*5*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1))
h_fc1_drop = tf.nn.dropout(h_fc1, rate=0.5)

In [16]:
W_fc2 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1))
# b_fc2 = tf.constant(0.1, shape=[10])

h = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2)) # + b_fc2)

## 훈련 정의

In [17]:
cross_entropy = -tf.reduce_sum(y*tf.log(h), reduction_indices=[1])
loss = tf.reduce_mean(cross_entropy)
train = tf.train.AdamOptimizer(0.001).minimize(loss)
# GradientDescentOptimizer는 학습률이 높아도 괜찮지만(최저점만나면 멈춤), 
# AdamOptimizer는 학습률이 너무 높은 것은 좋지 않음(확산률이 높아짐, 최저점 만나도 그방향으로 계속 지속해서 NaN 등이 발생할 수 있음)
# 둘의 기본 Learning Rate 크기가 다르기 때문에 주의해야 함.

## 세션 초기화 및 학습

In [18]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [19]:
tot_cnt = len(train_X)

for epoch in range(40):
    avg_loss = 0
    batch_size = 128
    tot_iter = int(tot_cnt / batch_size)
    for step in range(tot_iter):
        batch_X = train_X[int(step*batch_size):int((step+1)*batch_size)]
        batch_y = train_y[int(step*batch_size):int((step+1)*batch_size)]
        _, loss_ = sess.run([train, loss], feed_dict={x:batch_X, y:batch_y})
        avg_loss += (loss_ / tot_iter)
        print("{:03d} - {:f}".format(step, loss_), end="\r")
    if tot_cnt//tot_iter != 0:
        batch_X = train_X[int(tot_iter*batch_size):]
        batch_y = train_y[int(tot_iter*batch_size):]
        _, loss_ = sess.run([train, loss], feed_dict={x:batch_X, y:batch_y})
        avg_loss += (loss_ / tot_iter)
        print("{:03d} - {:f}".format(step, loss_), end="\r")
    if epoch%100 == 0:
        print("step {}, train loss {}".format(epoch, avg_loss))

step 0, train loss 1.3935686283641397


In [20]:
correct_prediction = tf.equal(tf.argmax(h, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x:test_X, y:test_y}))

0.97533333


In [21]:
sess.close()

# Keras

## 입력 데이터

In [23]:
import cv2
img = cv2.imread("images/digits.png", cv2.IMREAD_GRAYSCALE)

In [24]:
cells = [np.hsplit(row,100) for row in np.vsplit(img,50)] 
x = np.array(cells) / 255
# keras에서는 tensorflow를 섞어쓸 수 없기 때문에(tf.reshape 사용할 수 없음) 다음과 같이 shape를 정의
img_rows, img_cols = 20, 20
X = x.reshape(-1, img_rows, img_cols, 1) # numpy reshape 사용
# X = x[:,:].reshape(-1, 400).astype(np.float32)
y = pd.get_dummies(np.repeat(np.arange(10), 500)).to_numpy()

In [25]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=1)

In [26]:
train_X.shape, train_y.shape

((3500, 20, 20, 1), (3500, 10))

## 레이어 정의

In [27]:
from tensorflow.keras.models import Sequential
model = Sequential()

from tensorflow.keras.layers import Input, Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
model.add(Conv2D(32, kernel_size=(3, 3),
                 input_shape=(img_rows, img_cols, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) # stride 크기:2*2
model.add(Dropout(0.1))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


## 학습

In [28]:
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(0.001),
              metrics=['accuracy'])

In [29]:
model.fit(train_X, train_y, validation_data=(test_X, test_y),
          batch_size=128, epochs=200, verbose=1)
# 모든 test 이미지가 훈련 데이터에 맞추어져 있다는 보장은 없음
# > fit_generator 사용

Train on 3500 samples, validate on 1500 samples
Epoch 1/200
3500/3500 [==============================] - 1s 278us/sample - loss: 2.0305 - acc: 0.3406 - val_loss: 1.3139 - val_acc: 0.7480
Epoch 2/200
3500/3500 [==============================] - 1s 224us/sample - loss: 0.9859 - acc: 0.6997 - val_loss: 0.4299 - val_acc: 0.8820
Epoch 3/200
3500/3500 [==============================] - 1s 216us/sample - loss: 0.5241 - acc: 0.8417 - val_loss: 0.2687 - val_acc: 0.9200
Epoch 4/200
3500/3500 [==============================] - 1s 217us/sample - loss: 0.3791 - acc: 0.8863 - val_loss: 0.2260 - val_acc: 0.9333
Epoch 5/200
3500/3500 [==============================] - 1s 214us/sample - loss: 0.3291 - acc: 0.9020 - val_loss: 0.1962 - val_acc: 0.9427
Epoch 6/200
3500/3500 [==============================] - 1s 212us/sample - loss: 0.2799 - acc: 0.9137 - val_loss: 0.1725 - val_acc: 0.9427
Epoch 7/200
3500/3500 [==============================] - 1s 223us/sample - loss: 0.2402 - acc: 0.9283 - val_loss: 0.16

3500/3500 [==============================] - 1s 227us/sample - loss: 0.0290 - acc: 0.9917 - val_loss: 0.1056 - val_acc: 0.9713
Epoch 60/200
3500/3500 [==============================] - 1s 224us/sample - loss: 0.0269 - acc: 0.9917 - val_loss: 0.1037 - val_acc: 0.9747
Epoch 61/200
3500/3500 [==============================] - 1s 224us/sample - loss: 0.0266 - acc: 0.9909 - val_loss: 0.1113 - val_acc: 0.9740
Epoch 62/200
3500/3500 [==============================] - 1s 223us/sample - loss: 0.0242 - acc: 0.9934 - val_loss: 0.1094 - val_acc: 0.9733
Epoch 63/200
3500/3500 [==============================] - 1s 224us/sample - loss: 0.0349 - acc: 0.9880 - val_loss: 0.1032 - val_acc: 0.9727
Epoch 64/200
3500/3500 [==============================] - 1s 221us/sample - loss: 0.0283 - acc: 0.9920 - val_loss: 0.1043 - val_acc: 0.9740
Epoch 65/200
3500/3500 [==============================] - 1s 220us/sample - loss: 0.0281 - acc: 0.9911 - val_loss: 0.0975 - val_acc: 0.9760
Epoch 66/200
3500/3500 [=========

3500/3500 [==============================] - 1s 236us/sample - loss: 0.0153 - acc: 0.9937 - val_loss: 0.0954 - val_acc: 0.9787
Epoch 118/200
3500/3500 [==============================] - 1s 290us/sample - loss: 0.0136 - acc: 0.9960 - val_loss: 0.1050 - val_acc: 0.9767
Epoch 119/200
3500/3500 [==============================] - 1s 250us/sample - loss: 0.0172 - acc: 0.9949 - val_loss: 0.1111 - val_acc: 0.9760
Epoch 120/200
3500/3500 [==============================] - 1s 285us/sample - loss: 0.0155 - acc: 0.9937 - val_loss: 0.1170 - val_acc: 0.9760
Epoch 121/200
3500/3500 [==============================] - 1s 269us/sample - loss: 0.0106 - acc: 0.9966 - val_loss: 0.1094 - val_acc: 0.9767
Epoch 122/200
3500/3500 [==============================] - 1s 265us/sample - loss: 0.0085 - acc: 0.9974 - val_loss: 0.1136 - val_acc: 0.9793
Epoch 123/200
3500/3500 [==============================] - 1s 253us/sample - loss: 0.0115 - acc: 0.9969 - val_loss: 0.1238 - val_acc: 0.9787
Epoch 124/200
3500/3500 [==

3500/3500 [==============================] - 1s 216us/sample - loss: 0.0077 - acc: 0.9977 - val_loss: 0.1183 - val_acc: 0.9773
Epoch 176/200
3500/3500 [==============================] - 1s 207us/sample - loss: 0.0078 - acc: 0.9974 - val_loss: 0.1263 - val_acc: 0.9780
Epoch 177/200
3500/3500 [==============================] - 1s 226us/sample - loss: 0.0082 - acc: 0.9974 - val_loss: 0.1254 - val_acc: 0.9793
Epoch 178/200
3500/3500 [==============================] - 1s 211us/sample - loss: 0.0101 - acc: 0.9969 - val_loss: 0.1123 - val_acc: 0.9793
Epoch 179/200
3500/3500 [==============================] - 1s 209us/sample - loss: 0.0079 - acc: 0.9983 - val_loss: 0.1199 - val_acc: 0.9793
Epoch 180/200
3500/3500 [==============================] - 1s 222us/sample - loss: 0.0059 - acc: 0.9980 - val_loss: 0.1168 - val_acc: 0.9807
Epoch 181/200
3500/3500 [==============================] - 1s 208us/sample - loss: 0.0062 - acc: 0.9986 - val_loss: 0.1308 - val_acc: 0.9807
Epoch 182/200
3500/3500 [==

## 평가

In [31]:
model.evaluate(test_X, test_y, verbose=0)

[0.13704303781346727, 0.976]